# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900299  89.141418  130.451974  89.429260  127.713291   
2022-12-27  128.097473  87.275421  129.456968  88.833733  126.806945   
2022-12-28  124.166794  85.816360  129.082630  87.861023  123.999322   
2022-12-29  127.683739  88.287819  128.540804  88.699730  125.831682   
2022-12-30  127.998970  88.069481  128.018677  88.168735  125.536133   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  86.967727  128.974267  86.967727  63814900  17815000  
2022-12-27  86.883365  129.427415  88.645145  69007800  15470900  
2022-12-28  85.727034  127.742842  86.848619  85438400  17879600  
2022-12-29  86.342411  126.087812  86.382114  75703700  18280700  
2022-12-30  86.382131  126.501571  86.714637  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

result.head()

Strategy started!
Bought 1239 AAPL and 946 GOOG
Strategy finished!


,BALANCE,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED_TOTAL,EQUITY
ITERATION,,,,
2018-01-02,16.901165,99983.098835,0,100000.000000
2018-01-03,16.901165,100795.044468,0,100811.945633
2018-01-04,16.901165,101211.196243,0,101228.097408
2018-01-05,16.901165,102526.019138,0,102542.920303
2018-01-08,16.901165,102558.539791,0,102575.440956


## <b> Backtest Statistics </b>

In [5]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 16.90
Final equity: 241921.35
---------------------------


In [6]:
bt.get_trade_statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'mean_profit': 141921.35396194458,
 'median_profit': 141921.35396194458,
 'mean_profit_win': 141921.35396194458,
 'mean_profit_loss': nan,
 'median_profit_win': 141921.35396194458,
 'median_profit_loss': nan,
 'std_profit': nan,
 'mad_profit': 0.0,
 'skewness': nan,
 'kurtosis': nan,
 'mean_median_gap': 0.0,
 'mean_profit_over_vol': nan,
 'median_profit_over_vol': nan}

In [7]:
bt.get_equity_statistics_dollar()

{'total_profit': 141921.35396194458,
 'mean_profit': 112.81506674240428,
 'median_profit': 256.19779205322266,
 'std_profit': 4002.754669443937,
 'mad_profit': 2424.4263834964577,
 'skewness': -0.1935027312495447,
 'kurtosis': 3.0569025302745834,
 'mean_median_gap': -143.3827253108184,
 'mean_profit_over_vol': 0.02818435703881811,
 'median_profit_over_vol': 0.06400536960433142,
 'max_drawdown': -122839.07774353027}

In [8]:
bt.get_equity_statistics_return()

{'total_return': 1.4192135396194456,
 'mean_return': 0.0008838019357975216,
 'median_return': 0.0016636933631126727,
 'std_return': 0.019027788295764448,
 'mad_return': 0.01407427715282407,
 'skewness': -0.1545748678997562,
 'kurtosis': 4.071156728985332,
 'mean_median_gap': -0.000779891427315151,
 'mean_return_over_vol': 0.04644795927197983,
 'median_return_over_vol': 0.08743493133581941,
 'max_drawdown': -0.3432400922270106}

In [9]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,REASON_CLOSED,POSITIONS_FILLED,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED,POSITIONS_UNFILLED_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE,AGE
0,49edd8fe-69d1-4f2c-baff-fbfb7db3de3b,OPEN,Buy and Hold Trade,141921.353962,2018-01-02,None,None,"{'AAPL': {'quantity': 1239, 'value': 158590.72...",241904.452797,{},0,None,None,None,1823 days


In [10]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,TRADE_ID,ITERATION_REQUESTED,QUANTITY_REQUESTED,PRICE_REQUESTED,FEE_REQUESTED,GROSS_TOTAL_REQUESTED,TOTAL_REQUESTED,ITERATION_FILLED,QUANTITY_FILLED,PRICE_FILLED,FEE_FILLED,GROSS_TOTAL_FILLED,TOTAL_FILLED
0,c670f548-b789-43f0-a98d-33007aaae288,AAPL,BUY,FILLED,49edd8fe-69d1-4f2c-baff-fbfb7db3de3b,2018-01-02,1239,40.341892,0.0,49983.604488,49983.604488,2018-01-02,1239,40.341892,0.0,-49983.604488,-49983.604488
1,04c65a7e-7615-42d2-8e77-23a8592384f6,GOOG,BUY,FILLED,49edd8fe-69d1-4f2c-baff-fbfb7db3de3b,2018-01-02,946,52.853588,0.0,49999.494347,49999.494347,2018-01-02,946,52.853588,0.0,-49999.494347,-49999.494347


In [11]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.72386932373)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.7289276123)}}

In [12]:
bt.strategy.positions_unfilled

{}

In [13]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.72386932373)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.7289276123)}}